In [ ]:
import sys, os
import glob
import weakref

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import tqdm

from bio2byte.mdutils.utils.BBanalysis import BBDihedrals, stack_uneven, boltzmann_energy

# --- Font size and familiy
matplotlib.rcParams['font.size']          =   10
matplotlib.rcParams['axes.labelsize']     =   10
#matplotlib.rcParams['font.serif']         =   ["Cambria"]
#matplotlib.rcParams['font.sans-serif']    =   ["Arial"]
#matplotlib.rcParams['axes.unicode_minus'] =   False # Use for Noto Fonts
matplotlib.rcParams["mathtext.fontset"]   =    "dejavusans" # "dejavusans" "dejavuserif" "stix"
matplotlib.rcParams['axes.linewidth']     =   1
matplotlib.rcParams['lines.linewidth']    =   .9

# Read dihedral data

* DO NOT execute, unless you need to read in new data

In [ ]:
residues = sorted([
    "ALA", "ARG", "ASH", "ASN", "ASP", 
    "CYS", "GLH", "GLN", "GLU", "GLY", 
    "HIP", "HIS", "ILE", "LEU", "LYS", 
    "MET", "PHE", "PRO", "PTR", "S1P",
    "SEP", "SER", "T1P", "THR", "TPO", 
    "TRP", "TYR", "VAL", "Y1P"])

In [ ]:
for plength in [6,7,8,9]:
    
    g = "g" * (plength - 6)
    
    print(f"Loading {plength}-peptides ", end="")
    peptide_names = []
    peptide_data  = []
    for pname in tqdm.tqdm([f"gg{x+g+y}gg" for x in residues for y in residues]):
        bbobj = BBDihedrals(f"../{plength}peptides/{pname}/analy/dihedrals.csv")
        bbobj.histogram_bins = np.arange(-180,181,9)
        peptide_names.append(pname)
        peptide_data.append(bbobj)

    n = len(peptide_names)
    diffMtx = np.zeros((n, n), dtype=np.float32)

    print("Calculating RMSD...")
    fn = lambda x: (x**2 - x) // 2 # CumSum for arithmetic progression a(x) = x-1
    with tqdm.tqdm(total=fn(len(peptide_data))) as progbar:
        for i, bbobj1 in enumerate(peptide_data):
            for j, bbobj2 in enumerate(peptide_data[i+1:], start=i+1):
                delta = np.stack([(bbobj1.hist[i] - bbobj2.hist[i]) **2 for i in bbobj1.iterresids()], 
                                 axis=2)
                rmsd  = np.sqrt(np.mean(delta))
                diffMtx[i,j] = rmsd
                diffMtx[j,i] = rmsd
                progbar.update(1)

    np.save(f"rms/rms{plength}peptides.npy", diffMtx, allow_pickle=True)

# Plot RMS Matrix


## :: Normalization

In [ ]:
i,j = 0.,0.
for npyfile in glob.glob("rms/rms*peptides.npy"):
    arr = np.load(npyfile)
    i = min(i, np.min(arr))
    j = max(j, np.max(arr))
norma   = matplotlib.colors.Normalize(i,j)
del arr

cmap = "viridis"

## :: 6-peptides

In [ ]:
for i in np.arange(6) + 2:
    diffMtx = np.load(f"rms_6peptides_resid{i:d}.npy")
    n = int(diffMtx.shape[0] ** .5)

    fig, ax = plt.subplots(figsize=(10,9), dpi=600)

    ax.grid(True, "minor", "both", color="w", linewidth=.6, alpha=.5)
    hmap = ax.imshow(diffMtx, cmap=cmap, norm=norma, extent=(0,n**2,n**2,0))
    cbar = plt.colorbar(hmap, ax=ax, shrink=.7)

    ax.xaxis.set_tick_params(which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
    ax.set_xticks(np.arange(n/2, n**2, n)); ax.set_xticklabels(residues, rotation=-90)
    ax.set_xticks(np.arange(0, n**2, n/2), minor=True)
    ax.set_yticks(np.arange(n/2, n**2, n)); ax.set_yticklabels(residues)
    ax.set_yticks(np.arange(0, n**2, n/2), minor=True)

    fig.savefig(f"figures/rms_6peptides_resid{i:d}.png", dpi=600, bbox_inches='tight')
    plt.clf()

## :: 7-peptides

In [ ]:
for i in np.arange(7) + 2:
    diffMtx = np.load(f"rms_7peptides_resid{i:d}.npy")
    n = int(diffMtx.shape[0] ** .5)

    fig, ax = plt.subplots(figsize=(10,9), dpi=600)

    ax.grid(True, "minor", "both", color="w", linewidth=.6, alpha=.5)
    hmap = ax.imshow(diffMtx, cmap=cmap, norm=norma, extent=(0,n**2,n**2,0))
    cbar = plt.colorbar(hmap, ax=ax, shrink=.7)

    ax.xaxis.set_tick_params(which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
    ax.set_xticks(np.arange(n/2, n**2, n)); ax.set_xticklabels(residues, rotation=-90)
    ax.set_xticks(np.arange(0, n**2, n/2), minor=True)
    ax.set_yticks(np.arange(n/2, n**2, n)); ax.set_yticklabels(residues)
    ax.set_yticks(np.arange(0, n**2, n/2), minor=True)

    fig.savefig(f"rms_7peptides_resid{i:d}.png", dpi=600, bbox_inches='tight')
    plt.clf()

## :: 8-peptides

In [ ]:
for i in np.arange(8) + 2:
    diffMtx = np.load(f"rms_8peptides_resid{i:d}.npy")
    n = int(diffMtx.shape[0] ** .5)

    fig, ax = plt.subplots(figsize=(10,9), dpi=600)

    ax.grid(True, "minor", "both", color="w", linewidth=.6, alpha=.5)
    hmap = ax.imshow(diffMtx, cmap=cmap, norm=norma, extent=(0,n**2,n**2,0))
    cbar = plt.colorbar(hmap, ax=ax, shrink=.7)

    ax.xaxis.set_tick_params(which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
    ax.set_xticks(np.arange(n/2, n**2, n)); ax.set_xticklabels(residues, rotation=-90)
    ax.set_xticks(np.arange(0, n**2, n/2), minor=True)
    ax.set_yticks(np.arange(n/2, n**2, n)); ax.set_yticklabels(residues)
    ax.set_yticks(np.arange(0, n**2, n/2), minor=True)

    fig.savefig(f"rms_8peptides_resid{i:d}.png", dpi=600, bbox_inches='tight')
    plt.clf()

## :: 9peptides

In [ ]:
for i in np.arange(9) + 2:
    diffMtx = np.load(f"rms_9peptides_resid{i:d}.npy")
    n = int(diffMtx.shape[0] ** .5)

    fig, ax = plt.subplots(figsize=(10,9), dpi=600)

    ax.grid(True, "minor", "both", color="w", linewidth=.6, alpha=.5)
    hmap = ax.imshow(diffMtx, cmap=cmap, norm=norma, extent=(0,n**2,n**2,0))
    cbar = plt.colorbar(hmap, ax=ax, shrink=.7)

    ax.xaxis.set_tick_params(which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
    ax.set_xticks(np.arange(n/2, n**2, n)); ax.set_xticklabels(residues, rotation=-90)
    ax.set_xticks(np.arange(0, n**2, n/2), minor=True)
    ax.set_yticks(np.arange(n/2, n**2, n)); ax.set_yticklabels(residues)
    ax.set_yticks(np.arange(0, n**2, n/2), minor=True)

    fig.savefig(f"rms_9peptides_resid{i:d}.png", dpi=600, bbox_inches='tight')
    plt.clf()